In [10]:
import psycopg2
import pandas as pd

# Database connection parameters
db_params = {
    'dbname': 'air_data',
    'user': 'admin',
    'password': 'adminpassword',
    'host': 'localhost',
    'port': '54329'
}

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# SQL query to create the new table
create_table_query = """
CREATE TABLE IF NOT EXISTS aqi_summary (
    id SERIAL PRIMARY KEY,
    uid INTEGER,
    current_aqi INTEGER,
    avg_aqi_last_hour FLOAT,
    avg_aqi_last_day FLOAT,
    latitude FLOAT,
    longitude FLOAT,
    current_aqi_timestamp TIMESTAMP
);
"""

# Execute the create table query
cursor.execute(create_table_query)
conn.commit()

In [13]:
# SQL query to select data for the new table
select_data_query = """
SELECT
    aqi_data.uid,
    aqi_data.aqi AS current_aqi,
    (SELECT AVG(sub.aqi)
        FROM aqi_data sub
        WHERE sub.uid = aqi_data.uid
        AND sub.timestamp >= aqi_data.timestamp - INTERVAL '1 hour'
        AND sub.timestamp <= aqi_data.timestamp
    ) AS avg_aqi_last_hour,
    (SELECT AVG(sub.aqi)
        FROM aqi_data sub
        WHERE sub.uid = aqi_data.uid
        AND sub.timestamp >= aqi_data.timestamp - INTERVAL '1 day'
        AND sub.timestamp <= aqi_data.timestamp
    ) AS avg_aqi_last_day,
    station_info.latitude,
    station_info.longitude,
    aqi_data.timestamp AS current_aqi_timestamp
FROM
    aqi_data
JOIN
    station_info ON aqi_data.uid = station_info.uid
WHERE
    aqi_data.timestamp = (
        SELECT MAX(timestamp)
        FROM aqi_data AS sub
        WHERE sub.uid = aqi_data.uid
    );
"""

# SQL query to insert data into the new table
insert_data_query = """
"""

# Execute the select data query
cursor.execute(select_data_query)
result = cursor.fetchall()

# Convert the result to a pandas DataFrame for easier inspection
columns = [desc[0] for desc in cursor.description]
df = pd.DataFrame(result, columns=columns)

# Display the DataFrame
print(df)

# # Execute the insert data query
# cursor.execute(insert_data_query)
# conn.commit()

        uid  current_aqi     avg_aqi_last_hour      avg_aqi_last_day  \
0   -419611         52.0   52.0000000000000000   52.0000000000000000   
1   -461407         17.0   17.0000000000000000   17.0000000000000000   
2   -459532        167.0  167.0000000000000000  167.0000000000000000   
3   -421555        189.0  189.0000000000000000  189.0000000000000000   
4   -419767          9.0    9.0000000000000000    9.0000000000000000   
..      ...          ...                   ...                   ...   
346    1821        110.0  110.0000000000000000  110.0000000000000000   
347    9013        106.0  103.0000000000000000  103.6250000000000000   
348    1823        100.0   99.0000000000000000   91.1428571428571429   
349   13026        100.0  103.0000000000000000  120.1666666666666667   
350    9014        139.0  136.0000000000000000  122.6153846153846154   

      latitude   longitude current_aqi_timestamp  
0     6.554985  101.302274   2025-02-22 11:29:55  
1     7.973226   99.010562   2025

In [ ]:
# Close the cursor and connection
cursor.close()
conn.close()